In [ ]:
import spacy
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
pride_df = pd.read_csv('../input/pride-prejudice-clean-dataset/pride.csv', delimiter = ',')
pride_df

In [ ]:
def cleaning(doc):
    txt = [token.lemma_.lower() for token in doc if not token.is_stop and len(token) > 1] # this line tokenizes, takes out stopwords, and returns lemmas. We also lowercase everything (works for english, not necessarily other langauges) and also only take words with more than one letter to get rid of punctuation. There's smarter ways to do that -- but this is meant to be simplistic!!
    if len(txt) > 2: # dump any sentences with less than 2 words -- word2vec is based on collocation, after all...
        return " ".join(txt)

def process(df):
    docs = [row for row in pride_df["Pride and Prejudice (Novel)"]] # get your documents as a list of individual texts -- if you get a KeyError you might have to check your dataframe
    txts = [cleaning(doc) for doc in nlp.pipe(docs, batch_size=500, n_process = -1)] # nlp.pipe is some spacy magic -- this is where the processing happens
    sentences = [row.split() for row in txts if row != None] # we need to return the cleaned data (from cleaning(doc)) as sentences for Gensim
    return sentences # this is what we pass to Gensim

In [ ]:
nlp = spacy.load("en_core_web_sm", disable = ["ner","parser"])

## now process our data
sentences = process(pride_df)

In [ ]:
## first we initialize the model
from gensim.models import Word2Vec

model = Word2Vec(
    min_count = 1, ## you can mess around with these parameters (don't mess with workers though!); min_count refers to words that appear at least N times 
    window = 4, ## window refers to the size of the window +/- N words for something to count as a collocation
    workers = 4 ## leave this one alone...refers to the number of CPUs/threads to use. This works with Google Colab so just leave it alone for now. 
)

model.build_vocab(sentences, progress_per = 50) # remember that sentences is just what we output after spacy did its work in the process(df) function

In [ ]:
## now we train the model on our data

model.train(sentences, total_examples = model.corpus_count, epochs = 30)

## This might take quite some time again

In [ ]:
# first, save the model

model_savename = "pride.w2v"
model.wv.save(model_savename)

In [ ]:
## ok now the model is saved, so lets reload it
## we need to import a helper function from Gensim to reload the model

from gensim.models import KeyedVectors

model = KeyedVectors.load("./pride.w2v")

In [ ]:
model.vocab # generate list of vocabularies. This will work in Jupyter notebook

In [ ]:
## let's make a table of our vocabulary with frequency

records = list()

## iterate over every word in the vocab and get its frequency and save it to records
for word in model.vocab:
    records.append((word, model.vocab[word].count))

## make a table with some pandas magic
model_vocab_df = pd.DataFrame.from_records(records, columns = ["lemma","frequency"])
model_vocab_df.sort_values("frequency", ascending=False)

In [ ]:
def naive_projection(x_axis, y_axis, test_words, model, plot_size=10):


  ## check if you have the right input data
    if len(x_axis) != 2:
        print("You must only have two antonyms in your x-axis")
    elif len(y_axis) !=2:
        print("You must only have two antonyms in your y-axis")
    else:

    ## lets do the projection
        x = list() # store x values for each test word
        y = list()

    ## now, we calculate the x and y coordinates for each of our test words
    for word in test_words:
        x_val = model.distance(x_axis[0], word) - model.distance(x_axis[1], word) # x_axis[0] is the first word of your antonym pair, x_axis[1] is the second word
        y_val = model.distance(y_axis[0], word) - model.distance(y_axis[1], word) # same thing, 0,1 here to select your antonyms
        x.append(x_val) # save x/y in our lists above
        y.append(y_val)
    
    ## now we plot the x/y values we just calculated

    fig, ax = plt.subplots(figsize=(plot_size,plot_size))

    for i in range(len(x)):
        ax.scatter(x[i], y[i])
        ax.annotate(test_words[i], (x[i], y[i]))
    
    xlab = x_axis[0] + " --- " + x_axis[1]
    ylab = y_axis[0] + " --- " + y_axis[1]
    plt.xlabel(xlab)
    plt.ylabel(ylab)

    plt.show()

In [ ]:
x_axis = ["husband", "wife"]
y_axis = ["rich","poor"]
test_words = ["elizabeth", "bennet", "fitzwilliam", "darcy", "bourgh", "jane", "bingley", "caroline", "wickham", "lydia", "gardiner", "william", "charlotte", "georgiana", "mary", "catherine"] 

naive_projection(x_axis, y_axis, test_words, model)

In [ ]:
x_axis = ["man","woman"]
y_axis = ["elegant","disgrace"]
test_words = ["elizabeth", "bennet", "fitzwilliam", "darcy", "bourgh", "jane", "bingley", "caroline", "wickham", "lydia", "gardiner", "william", "charlotte", "georgiana", "mary", "catherine"] 

naive_projection(x_axis, y_axis, test_words, model)

In [ ]:
x_axis = ["father","mother"]
y_axis = ["master","servant"]
test_words = ["elizabeth", "bennet", "fitzwilliam", "darcy", "bourgh", "jane", "bingley", "caroline", "wickham", "lydia", "gardiner", "william", "charlotte", "georgiana", "mary", "catherine"] 

naive_projection(x_axis, y_axis, test_words, model)

In [ ]:
x_axis = ["gentleman", "lady"]
y_axis = ["fortune", "debt"]
test_words = ["elizabeth", "bennet", "fitzwilliam", "darcy", "bourgh", "jane", "bingley", "caroline", "wickham", "lydia", "gardiner", "william", "charlotte", "georgiana", "mary", "catherine"] 

naive_projection(x_axis, y_axis, test_words, model)

In [ ]:
def advanced_projection(x_dimensions, y_dimensions, test_words, model, plot_size=8, xlab="label", ylab = "label"):
    x = list()
    y = list()

    for word in test_words:
        x_vals = list()
        y_vals = list()

        for dim in x_dimensions:
            xval = model.distance(dim[0], word) - model.distance(dim[1], word)
            x_vals.append(xval)
    ## repeat for y values
        for dim in y_dimensions:
            yval = model.distance(dim[0], word) - model.distance(dim[1], word)
            y_vals.append(yval)

    ## ok now we need to take the average of all the x_vals and y_vals we collected for this word
        xavg = statistics.mean(x_vals)
        yavg = statistics.mean(y_vals)

    ## now lets save this to our x and y lists that we set up above (outside the for word in test_words loop) so that we can plot the word
        x.append(xavg)
        y.append(yavg)


    
    fig, ax = plt.subplots(figsize=(plot_size,plot_size))
    for i in range(len(x)):
        ax.scatter(x[i], y[i])
        ax.annotate(test_words[i], (x[i], y[i]))
        
    plt.xlabel(xlab)
    plt.ylabel(ylab)
    plt.show()
    plt.savefig('pride_scatter_book.png')
    
#     fig, ax = plt.subplots(figsize=(plot_size,plot_size))
#     for i in range(len(x)):
#         ax.bar(x[i], y[i])
#         ax.annotate(test_words[i], (x[i], y[i]))
        
#     plt.xlabel(xlab)
#     plt.ylabel(ylab)
#     plt.show()
#     plt.savefig('pride_bar_book.png')
    
#     plt.figure()
#     plt.bar(monthly.index, monthly.price, color = 'green')

#     x = plt.gca().xaxis
#     for item in x.get_ticklabels(): # rotate tick labels
#         item.set_rotation(45)

In [ ]:
import statistics

x_dimensions =  [
                ["husband", "wife"],
                ["man","woman"],
                ["father","mother"],
                ["gentleman", "lady"],
                ["mr", "miss"],
                ["mr", "mrs"],
                ["brother", "sister"],
                ["sir", "madam"]
]


y_dimensions = [
                ["rich","poor"],
                ["master","servant"],
                ["fortune", "misfortune"],
                ["wealth", "poverty"]
]


test_words = ["elizabeth", "bennet", "fitzwilliam", "darcy", "bourgh", "jane", "bingley", "caroline", "wickham", "lydia", "gardiner", "william", "charlotte", "georgiana", "mary", "catherine"] 
xlab = "maleness -- femaleness"
ylab = "richness -- poorness"

advanced_projection(
    x_dimensions,
    y_dimensions,
    test_words,
    model,
    plot_size=10,
    xlab = xlab,
    ylab = ylab)